# # ⛈️ Analisis Data Eksploratif: Dampak Cuaca terhadap Lalu Lintas Kota Medan
#
# **Tujuan Notebook:**
# Notebook ini bertujuan untuk menganalisis secara mendalam bagaimana berbagai kondisi cuaca (curah hujan, suhu, jarak pandang, kelembaban) memengaruhi dinamika lalu lintas di Kota Medan, khususnya terhadap kecepatan rata-rata kendaraan.
#
# **Dataset yang Digunakan:** Data dari Silver Layer HDFS (gabungan trafik dan cuaca yang sudah dibersihkan dan di-join).
#
# **Tahapan Analisis:**
# 1. Setup Environment Spark & Pustaka
# 2. Pemuatan Data Gabungan (Trafik & Cuaca)
# 3. Pemeriksaan dan Persiapan Data Cuaca
# 4. Analisis Statistik Deskriptif Variabel Cuaca
# 5. Analisis Korelasi Antara Variabel Cuaca dan Kecepatan Kendaraan
# 6. Visualisasi Dampak Curah Hujan terhadap Kecepatan
# 7. Visualisasi Dampak Suhu terhadap Kecepatan
# 8. Visualisasi Dampak Jarak Pandang terhadap Kecepatan
# 9. Analisis Komparatif: Kondisi Lalu Lintas pada Cuaca Ekstrem vs. Normal
# 10. Kesimpulan Awal dan Implikasi untuk Pemodelan

In [ ]:
# 1. Setup Environment Spark & Pustaka
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, corr, when, round as spark_round
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

spark = SparkSession.builder \
    .appName("EDAWeatherImpactMedan") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

print("SparkSession berhasil dibuat!")

# ## 2. Pemuatan Data Gabungan (Trafik & Cuaca)
# Memuat data dari HDFS Silver Layer yang sudah berisi informasi trafik dan cuaca yang terpadu.


In [ ]:
silver_data_path = "/user/hive/warehouse/kemacetan_medan/silver/trafik_cuaca/data_gabungan.parquet"
try:
    df_merged = spark.read.parquet(silver_data_path)
    print(f"Data gabungan berhasil dimuat. Jumlah baris: {df_merged.count()}")
    df_merged.printSchema()
    # Tampilkan beberapa kolom relevan untuk cuaca dan kecepatan
    df_merged.select("timestamp", "speed_kmh", "rainfall_mm", "temperature_c", "humidity_percent", "visibility_km").show(5)
except Exception as e:
    print(f"Error memuat data: {e}")

# ## 3. Pemeriksaan dan Persiapan Data Cuaca
# Memastikan kolom-kolom cuaca memiliki tipe data yang benar dan menangani nilai-nilai yang mungkin tidak valid.


In [ ]:
# Kolom cuaca utama yang akan dianalisis
weather_cols = ["rainfall_mm", "temperature_c", "humidity_percent", "visibility_km"]

# Cek nilai null (seharusnya sudah ditangani di tahap Silver, tapi baik untuk verifikasi)
print("Jumlah nilai null pada kolom cuaca:")
df_merged.select([count(when(col(c).isNull(), c)).alias(c) for c in weather_cols]).show()

# Pastikan tipe data sudah numerik (Float atau Double) - seharusnya sudah dari proses Silver
for w_col in weather_cols:
    if dict(df_merged.dtypes)[w_col] not in ['float', 'double']:
        print(f"Peringatan: Kolom {w_col} bukan tipe numerik.")
        # Lakukan casting jika perlu, contoh: df_merged = df_merged.withColumn(w_col, col(w_col).cast("float"))

# Filter nilai ekstrem yang tidak realistis jika ada (misalnya, suhu di bawah -20 atau di atas 50 untuk Medan)
df_merged_filtered = df_merged.filter(
    (col("temperature_c") >= 15) & (col("temperature_c") <= 40) & \
    (col("humidity_percent") >= 10) & (col("humidity_percent") <= 100) & \
    (col("visibility_km") >= 0) & (col("rainfall_mm") >=0)
)
print(f"Jumlah baris setelah filter data cuaca: {df_merged_filtered.count()}")


# ## 4. Analisis Statistik Deskriptif Variabel Cuaca
# Memahami sebaran dan tendensi sentral dari setiap variabel cuaca.


In [ ]:
print("Statistik Deskriptif untuk Variabel Cuaca:")
df_merged_filtered.select(weather_cols).describe().show()

# ## 5. Analisis Korelasi Antara Variabel Cuaca dan Kecepatan Kendaraan
# Mengukur seberapa kuat hubungan linear antara setiap variabel cuaca dengan `speed_kmh`.


In [ ]:
print("Koefisien Korelasi Pearson antara Variabel Cuaca dan Kecepatan (speed_kmh):")
correlations = {}
for w_col in weather_cols:
    correlation_value = df_merged_filtered.stat.corr("speed_kmh", w_col)
    correlations[w_col] = correlation_value
    print(f"- Korelasi speed_kmh vs {w_col}: {correlation_value:.4f}")

# Visualisasi matriks korelasi (opsional, jika ingin melihat antar variabel cuaca juga)
# numeric_cols_for_corr = ["speed_kmh"] + weather_cols
# corr_matrix_pd = df_merged_filtered.select(numeric_cols_for_corr).toPandas().corr() # Hati-hati dengan toPandas() pada data besar
# plt.figure(figsize=(8, 6))
# sns.heatmap(corr_matrix_pd, annot=True, cmap='coolwarm', fmt=".2f")
# plt.title('Matriks Korelasi antara Kecepatan dan Variabel Cuaca')
# plt.show()

# **Interpretasi Korelasi:**
# * Curah hujan (`rainfall_mm`) menunjukkan korelasi [positif/negatif, kuat/lemah] dengan kecepatan. Nilai [X] menyiratkan bahwa semakin [tinggi/rendah] curah hujan, kecepatan cenderung [menurun/meningkat].
# * Suhu (`temperature_c`) menunjukkan korelasi [positif/negatif, kuat/lemah] ...
# * Jarak pandang (`visibility_km`) menunjukkan korelasi [positif/negatif, kuat/lemah] ...
# * (Komentari setiap korelasi)

# ## 6. Visualisasi Dampak Curah Hujan (`rainfall_mm`) terhadap Kecepatan

In [ ]:
# Untuk visualisasi yang lebih jelas, kita bisa mengkategorikan curah hujan
df_viz = df_merged_filtered.withColumn("kategori_hujan",
    when(col("rainfall_mm") == 0, "Tidak Hujan (0mm)")
    .when((col("rainfall_mm") > 0) & (col("rainfall_mm") <= 5), "Hujan Ringan (0-5mm)")
    .when((col("rainfall_mm") > 5) & (col("rainfall_mm") <= 15), "Hujan Sedang (5-15mm)")
    .otherwise("Hujan Lebat (>15mm)")
)

avg_speed_by_rainfall_cat = df_viz.groupBy("kategori_hujan") \
    .agg(avg("speed_kmh").alias("avg_speed"), count("*").alias("jumlah_observasi")) \
    .orderBy("avg_speed") \
    .toPandas()

plt.figure(figsize=(10, 6))
sns.barplot(data=avg_speed_by_rainfall_cat, x="kategori_hujan", y="avg_speed", palette="Blues_r",
            order=["Tidak Hujan (0mm)", "Hujan Ringan (0-5mm)", "Hujan Sedang (5-15mm)", "Hujan Lebat (>15mm)"])
plt.title('Dampak Intensitas Curah Hujan terhadap Kecepatan Rata-rata')
plt.xlabel('Kategori Curah Hujan')
plt.ylabel('Kecepatan Rata-rata (km/h)')
plt.xticks(rotation=15)
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.show()

print(avg_speed_by_rainfall_cat)

# **Interpretasi Dampak Curah Hujan:**
# * Terlihat jelas bahwa kecepatan rata-rata kendaraan menurun seiring dengan meningkatnya intensitas curah hujan.
# * Penurunan paling signifikan terjadi pada kategori [Hujan Lebat/Sedang].


# ## 7. Visualisasi Dampak Suhu (`temperature_c`) terhadap Kecepatan
# (Lakukan analisis serupa seperti pada curah hujan: scatter plot atau binning suhu dan lihat rata-rata kecepatan)


In [ ]:
# Contoh dengan binning suhu
df_viz = df_viz.withColumn("kategori_suhu",
    when(col("temperature_c") < 25, "Sejuk (<25°C)")
    .when((col("temperature_c") >= 25) & (col("temperature_c") < 30), "Hangat (25-30°C)")
    .otherwise("Panas (>=30°C)")
)

avg_speed_by_temp_cat = df_viz.groupBy("kategori_suhu") \
    .agg(avg("speed_kmh").alias("avg_speed")) \
    .orderBy("avg_speed") \
    .toPandas()

plt.figure(figsize=(10, 6))
sns.barplot(data=avg_speed_by_temp_cat, x="kategori_suhu", y="avg_speed", palette="YlOrRd")
plt.title('Dampak Suhu terhadap Kecepatan Rata-rata')
# ... (label dan styling lainnya)
plt.show()

# **Interpretasi Dampak Suhu:**
# * [Jelaskan pola yang terlihat, misalnya apakah suhu sangat panas atau sangat sejuk memengaruhi kecepatan]


# ## 8. Visualisasi Dampak Jarak Pandang (`visibility_km`) terhadap Kecepatan
# (Lakukan analisis serupa: scatter plot atau binning jarak pandang)


In [ ]:
# Scatter plot speed_kmh vs visibility_km (ambil sampel jika data besar)
visibility_speed_sample_pd = df_merged_filtered.select("visibility_km", "speed_kmh").sample(False, 0.05).toPandas()

plt.figure(figsize=(10,6))
sns.scatterplot(data=visibility_speed_sample_pd, x="visibility_km", y="speed_kmh", alpha=0.5)
plt.title("Hubungan Jarak Pandang dengan Kecepatan Kendaraan")
plt.xlabel("Jarak Pandang (km)")
plt.ylabel("Kecepatan (km/h)")
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# **Interpretasi Dampak Jarak Pandang:**
# * [Jelaskan pola yang terlihat, misalnya apakah jarak pandang rendah secara signifikan mengurangi kecepatan]


# ## 9. Analisis Komparatif: Kondisi Lalu Lintas pada Cuaca Ekstrem vs. Normal
# (Definisikan apa itu "cuaca ekstrem" misal hujan lebat & jarak pandang rendah, lalu bandingkan distribusi kecepatan atau kecepatan rata-rata dengan kondisi "normal")


In [ ]:
# Contoh definisi cuaca ekstrem
df_viz = df_viz.withColumn("kondisi_cuaca_ekstrem",
    when((col("rainfall_mm") > 10) & (col("visibility_km") < 2), "Ekstrem (Hujan Lebat & Vis Rendah)")
    .when((col("rainfall_mm") == 0) & (col("visibility_km") > 8), "Normal (Cerah & Vis Baik)")
    .otherwise("Lainnya")
)

# Ambil hanya kategori Ekstrem dan Normal untuk perbandingan yang jelas
df_comparison = df_viz.filter(col("kondisi_cuaca_ekstrem").isin(["Ekstrem (Hujan Lebat & Vis Rendah)", "Normal (Cerah & Vis Baik)"]))

plt.figure(figsize=(10,6))
sns.boxplot(data=df_comparison.toPandas(), x="kondisi_cuaca_ekstrem", y="speed_kmh", palette={"Normal (Cerah & Vis Baik)":"lightgreen", "Ekstrem (Hujan Lebat & Vis Rendah)":"salmon"})
plt.title("Perbandingan Distribusi Kecepatan: Cuaca Ekstrem vs. Normal")
# ... (label dan styling lainnya)
plt.show()

# **Interpretasi Perbandingan Cuaca Ekstrem vs. Normal:**
# * Boxplot dengan jelas menunjukkan bahwa pada kondisi cuaca ekstrem, median kecepatan [lebih rendah/sama/lebih tinggi] dan sebaran kecepatan menjadi [lebih sempit/lebar] dibandingkan kondisi normal.
# * Ini menguatkan dugaan bahwa cuaca ekstrem memiliki dampak signifikan terhadap kelancaran lalu lintas.


# ## 10. Kesimpulan Awal dan Implikasi untuk Pemodelan
# Berdasarkan analisis dampak cuaca:
# 1.  **Curah Hujan Signifikan**: Curah hujan, terutama intensitas sedang hingga lebat, secara konsisten menunjukkan dampak negatif terhadap kecepatan rata-rata kendaraan. Ini adalah kandidat fitur kuat.
# 2.  **Jarak Pandang Berpengaruh**: Penurunan jarak pandang juga berkorelasi dengan penurunan kecepatan, meskipun mungkin efeknya lebih terlihat pada kondisi ekstrem.
# 3.  **Suhu Kurang Konsisten (Contoh)**: Dampak suhu mungkin kurang langsung atau lebih kompleks, berinteraksi dengan faktor lain. (Sesuaikan dengan temuan Anda).
# 4.  **Implikasi untuk Fitur**: Variabel `rainfall_mm`, `visibility_km`, dan mungkin `temperature_c` (atau kategori darinya) perlu dipertimbangkan secara serius sebagai fitur dalam model prediksi kemacetan. Interaksi antara cuaca dan waktu (jam) juga bisa dieksplorasi.
# 5.  **Pentingnya Data Cuaca Akurat**: Kualitas prediksi kemacetan akan sangat bergantung pada ketersediaan dan akurasi data cuaca real-time atau prediksi cuaca jangka pendek.


In [ ]:
spark.stop()